In [ ]:
import time
time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime(105484586906))

'Sat, 03 Sep 5312 10:08:26 +0000'

##Practical Example of BPNN

credit: https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

In [ ]:
from random import seed
from random import random

from math import exp
import numpy as np
import pandas as pd


In [ ]:
 # Initialize a network, and randomly assign the weights
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network
 
seed(10)
network = initialize_network(2, 1, 1)
for layer in network:
	print(layer)

[{'weights': [0.5714025946899135, 0.4288890546751146, 0.5780913011344704]}]
[{'weights': [0.20609823213950174, 0.81332125135732]}]


In [ ]:
#manual assignment of weights
network = [[{'weights': [0.5, 0.1, 1]}],
 [{'weights': [1, -1]}]]

network

[[{'weights': [0.5, 0.1, 1]}], [{'weights': [1, -1]}]]

In [ ]:
#Forward propagation


# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Transfer neuron activation
def transfer(activation):
	return np.round(1.0 / (1.0 + exp(-activation)),2)

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs) #linear combination
			neuron['output'] = transfer(activation) # activation function e.g. sigmoid
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# test forward propagation
row = [1, 2]
output = forward_propagate(network, row)
for layer in network:
	print(layer)

[{'weights': [0.5, 0.1, 1], 'output': 0.85}]
[{'weights': [1, -1], 'output': 0.46}]


In [ ]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# test backpropagation of error

expected = [1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'weights': [0.5, 0.1, 1], 'output': 0.85, 'delta': 0.017102340000000004}]
[{'weights': [1, -1], 'output': 0.46, 'delta': 0.13413600000000003}]


In [ ]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']


lr = 0.5
update_weights(network, row, lr)

for layer in network:
	print(layer)

[{'weights': [0.50855117, 0.1, 1.00855117], 'output': 0.85, 'delta': 0.017102340000000004}]
[{'weights': [1.0570078, -0.932932], 'output': 0.46, 'delta': 0.13413600000000003}]


In [ ]:
#Apply this function to learn from data - you must provide the train variable which consist of your dataset

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs, val):
  
  for epoch in range(n_epoch+1):
    sum_error = 0
    
    for row in train:  
      outputs = forward_propagate(network, row[:5])
      expected = row[4:5]
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
    if epoch % 5 == 0:
       for row in val:  
         outputs = forward_propagate(network, row[:5])
         expected = row[4:5]
         sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
         backward_propagate_error(network, expected)
         update_weights(network, row, l_rate)
      
       print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
       
        
        


In [ ]:
# Test training backprop algorithm
seed(1)

dataset = pd.read_csv('train.csv')

dataset[["class"]] = dataset[["class"]].apply(lambda col:pd.Categorical(col).codes)
dataset = dataset.to_numpy().tolist()

val = pd.read_csv('val.csv')

val[["class"]] = val[["class"]].apply(lambda col:pd.Categorical(col).codes)
val = val.to_numpy().tolist()

test = pd.read_csv('test.csv')

test[["class"]] = test[["class"]].apply(lambda col:pd.Categorical(col).codes)
test = test.to_numpy().tolist()



# dataset = [[2.7810836,2.550537003,0],
# 	[1.465489372,2.362125076,0],
# 	[3.396561688,4.400293529,0],
# 	[1.38807019,1.850220317,0],
# 	[3.06407232,3.005305973,0],
# 	[7.627531214,2.759262235,1],
# 	[5.332441248,2.088626775,1],
# 	[6.922596716,1.77106367,1],
# 	[8.675418651,-0.242068655,1],
# 	[7.673756466,3.508563011,1]]


n_inputs = len(dataset[0]) - 1
n_hidden = 1
n_outputs = 1
network = initialize_network(n_inputs, n_hidden, n_outputs)


#use it if you want to determine initial weights:
# network = [[{'weights': [0.5, 0.1, 1]}],
#            [{'weights': [1, -1]}]]

epoch = 1000
lr = 0.5
train_network(network, dataset, lr, epoch, n_outputs, val)
# for layer in network:
# 	print(layer)

>epoch=0, lrate=0.500, error=10.563
>epoch=5, lrate=0.500, error=10.185
>epoch=10, lrate=0.500, error=10.171
>epoch=15, lrate=0.500, error=10.203
>epoch=20, lrate=0.500, error=10.203
>epoch=25, lrate=0.500, error=10.197
>epoch=30, lrate=0.500, error=10.197
>epoch=35, lrate=0.500, error=10.172
>epoch=40, lrate=0.500, error=10.185
>epoch=45, lrate=0.500, error=10.192
>epoch=50, lrate=0.500, error=10.191
>epoch=55, lrate=0.500, error=10.174
>epoch=60, lrate=0.500, error=8.644
>epoch=65, lrate=0.500, error=3.305
>epoch=70, lrate=0.500, error=1.407
>epoch=75, lrate=0.500, error=0.810
>epoch=80, lrate=0.500, error=0.552
>epoch=85, lrate=0.500, error=0.416
>epoch=90, lrate=0.500, error=0.321
>epoch=95, lrate=0.500, error=0.274
>epoch=100, lrate=0.500, error=0.223
>epoch=105, lrate=0.500, error=0.200
>epoch=110, lrate=0.500, error=0.169
>epoch=115, lrate=0.500, error=0.154
>epoch=120, lrate=0.500, error=0.139
>epoch=125, lrate=0.500, error=0.138
>epoch=130, lrate=0.500, error=0.120
>epoch=135,

In [ ]:
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs[0]

## Evaluation

In [ ]:
def validasi(dataset):
    actual = []
    preds = []
    for row in dataset:
        actual.append(row[-1])
        prediction = predict(network, row)
        preds.append(int(np.round(prediction)))
        print('Expected=%d, Got (actual)=%f, Got (prediction)=%d' % (row[-1], prediction, np.round(prediction)))

    result  = pd.DataFrame((actual,preds)).T
    result.columns  = ("actual", "preds")

    conf_matrix  =  pd.crosstab(result.actual, result.preds)
    conf_matrix

    tn  = conf_matrix.iloc[0,0]
    fp  = conf_matrix.iloc[0,1]
    fn  = conf_matrix.iloc[1,0]
    tp  = conf_matrix.iloc[1,1]

    accuracy =  (tp+tn)/(tp+tn+fp+fn) * 100
    recall =  tp/(tp+fn) * 100
    precision  = tp/(tp+fp) * 100
    f1Score = 2*(recall*precision)/(recall+precision)

    print ('accuracy =  %.3f' % accuracy)
    print ('recall =  %.3f'  % recall)
    print ('precision =  %.3f' % precision)
    print ('f1Score =  %.3f'  % f1Score)



In [ ]:
validasi(dataset)

Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.030000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.030000, Got (prediction)=0
Expected=0, Got (actual)=0.030000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=1, Got (actual)=0.990000, Got (prediction)=1
Expected=1, Got (actual)=0.990000, Got (prediction)=1
Expected=1, Got (actual)=0.990000, Got (prediction)=1
Expected=1, Got (actual)=0.990000, Got (prediction)=1
Expected=1, Got (actual)=0.9

In [ ]:
validasi(val)

Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.030000, Got (prediction)=0
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.990000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.990000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
accuracy =  100.000
recall =  100.000
precision =  100.000
f1Score =  100.000


In [ ]:
validasi(test)

Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.990000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.990000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
accuracy =  100.000
recall =  100.000
precision =  100.000
f1Score =  100.000


1. Ubah online learning menjadi batch learning
2. Modifikasi train network sehingga bisa menggunakan val data (per 5 epoch)
3. Evaluation menggunakan test data (bandingkan dengan accuracy training, validation)